In [2]:
import pickle
import pandas as pd
import math
import json
import numpy as np

In [3]:
data = json.load(open('/green-projects/project-sonyc_redhook/workspace/share/video_info_copy.json'))

File is structured with master keys being name of file, the values being other dictionaries, with inner keys being the frame number, and the value being an array of border coordinates and probability

Step 1: loop through dictionaries and make dataframe

In [4]:
data['/home/cusp/cmydlarz/redhook_video/data/1573276672.ts']

{'308': [[[18.5, 321.0, 191, 132]], [0.225791335105896]],
 '310': [[[18.5, 320.5, 193, 133]], [0.2184540182352066]],
 '630': [[[19.0, 316.5, 192, 139], [27.0, 340.5, 176, 115]],
  [0.5922264456748962, 0.5686070322990417]],
 '632': [[[12.5, 330.0, 203, 132], [24.5, 341.0, 181, 110]],
  [0.7574986815452576, 0.7172761559486389]]}

In [5]:
data['/home/cusp/cmydlarz/redhook_video/data/1573276672.ts']['308'][0][0][2] * data['/home/cusp/cmydlarz/redhook_video/data/1573276672.ts']['308'][0][0][3]

25212

In [6]:
start_timestamp = []
frame = []
actual_timestamp = []
area = []
probability = []

In [79]:
for master_key in data:
    for key in data[master_key]:
        start_timestamp.append(int(master_key[39:49]))
        frame.append(int(key))
        actual_timestamp.append(int(int(master_key[39:49]) + int(key)/15))
        area.append(data[master_key][key][0][0][2] * data[master_key][key][0][0][3])
        probability.append(data[master_key][key][1][0])

In [80]:
df = pd.DataFrame(data = {'start_timestamp': start_timestamp, 'frame': frame, \
                          'actual_timestamp': actual_timestamp, 'area': area, \
                          'probability': probability})

In [81]:
df.head()

,start_timestamp,frame,actual_timestamp,area,probability
0,1573276672,308,1573276692,25212,0.225791
1,1573276672,310,1573276692,25669,0.218454
2,1573276672,630,1573276714,26688,0.592226
3,1573276672,632,1573276714,26796,0.757499
4,1573061868,164,1573061878,25134,0.202466


In [82]:
len(df)

103710

In [83]:
max_set = []

In [84]:
def truck_filter(df, max_set, window_size, min_area, min_probability):
    """
    Returns an array of indices that most likely correspond to trucks, based on the probability value given by
    the image recognition algorithm.
    
    Parameters
    ----------
    df : dataframe
        The dataframe containing information gathered from an image recognition algorithm. Should contain 
        a starting timestamp column, a frame column, an area column, and a probability column.
        
    max_set : array
        The array to store the indices of likely instances of a truck.
        
    window_size : int
        The number of seconds before and after the timestamp of each row.
        
    min_area : int
        The minimum area of the bounding box. 
    
    min_probability : float
        The minimum probability given by the classifier.
        
    """
    for i in df.index:
        #Gets subsection of dataframe inside the window 
        min_timestamp = int(df['actual_timestamp'][i] - window_size/2)
#         print(str(i) + " " + str(min_timestamp))
        max_timestamp = int(df['actual_timestamp'][i] + window_size/2)
#         print(str(i) + " " + str(max_timestamp))
        window_df = df[(min_timestamp <= df['actual_timestamp']) & (max_timestamp >= df['actual_timestamp'])]

        #Checks conditions for probability, window size, etc
        if window_df['probability'].max() >= min_probability and window_df['area'].max() >= min_area:
            if(np.argmax(window_df['probability']) == i and i not in max_set):
                max_set.append(i)

In [85]:
truck_filter(df, max_set, 10, 80000, 0.7)

In [86]:
len(max_set)

7106

In [7]:
max_dataframe = df.iloc[max_set]

NameError: name 'df' is not defined

In [94]:
max_dataframe.tail()

,start_timestamp,frame,actual_timestamp,area,probability
103596,1573932897,364,1573932921,205440,0.944695
103660,1573931033,88,1573931038,55357,0.930733
103668,1573500724,504,1573500757,112950,0.898339
103674,1573921831,70,1573921835,280088,0.950858
103706,1573303753,88,1573303758,296973,0.978478


In [89]:
max_dataframe.to_csv('max_dataframe.csv')

In [1]:
max_dataframe['actual_timestamp']

NameError: name 'max_dataframe' is not defined

In [92]:
max_dataframe['actual_timestamp'].to_csv('max_timestamps.csv')

/home/cusp/meh708/.conda/envs/truck_update/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [93]:
len(max_dataframe)

7106